# 读取所有Chracter

In [1]:
from collections import defaultdict


in_path = '/ldev/tensorflow/nmt2/nmt/data/wordlevel/'

words_dict = {}
chars_dict = {}
for file in ['message','response']:
    characters = defaultdict(int)
    words = defaultdict(int)
    with open(in_path+'train.'+file,'r',encoding='utf-8') as fin:
        for line in fin.readlines():
            tokens = line.strip('\n').split()
            for token in tokens:
                words[token] += 1
                for char in token:
                    characters[char] += 1
        print(len(characters))
        print(len(words))
        words_dict[file] = words
        chars_dict[file] = characters
    
    

6688
120929
10549
618079


# 写一个程序，控制Character的数量
Character 包含双份的，即包含未终结的字符

In [1]:
def count_response(total_count,file,output_id):
    words = words_dict[file]
    chars = (chars_dict[file].keys())
    vocab = set()
    sorted_words = sorted(words.items(), key = lambda x:x[1], reverse=True)
    print('response char nums: %d' % len(chars) )
    for i in range(0,total_count - len(chars)):
        vocab.add(sorted_words[i][0])
    for char in chars:
            vocab.add(char)
    i = 0
    while(len(vocab) < total_count):
        vocab.add(sorted_words[i][0])
        i += 1
    if '@@' in vocab or ' ' in vocab:
        print('Error')
    with open('/ldev/tensorflow/nmt2/nmt/data/hybrid2/vocab.'+str(output_id)+'.'+file,'w+',encoding='utf-8') as fout:
        fout.write('\n'.join(['@@']+list(vocab)))
    print("response : %d" % len(vocab))

def count_message(total_count,file,output_id):
    words = words_dict[file]
    chars = (chars_dict[file].keys())
    vocab = set()
    sorted_words = sorted(words.items(), key = lambda x:x[1], reverse=True)
    print('message char nums: %d' % len(chars) )
    for i in range(total_count-len(chars)//4):
        vocab.add(sorted_words[i][0])   
    if '@@' in vocab or ' ' in vocab:
        print('Error')
    with open('/ldev/tensorflow/nmt2/nmt/data/hybrid2/vocab.'+str(output_id)+'.'+file,'w+',encoding='utf-8') as fout:
        fout.write('\n'.join(list(vocab)))
    print("message : %d" % len(vocab))


count_message(40000,'message',40000)
count_response(40000,'response',40000)
count_message(20000,'message',20000)
count_response(20000,'response',20000)
count_message(80000,'message',80000)
count_response(80000,'response',80000)

NameError: name 'words_dict' is not defined

# 读取对应的文件，然后生成相应的混合表示

In [8]:

in_path = '/ldev/tensorflow/nmt2/nmt/data/wordlevel/'
for file in ['response']:
    for vocab_size in [20000,40000,80000]:
        vocab = set()
        with open('/ldev/tensorflow/nmt2/nmt/data/hybrid2/vocab.'+str(vocab_size)+'.'+file,'r+',encoding='utf-8') as fin:
            for line in fin.readlines():
                vocab.add(line.strip())
        for prefix  in ['train','test','dev']:
            sen_len_counter = defaultdict(int)
            with open('/ldev/tensorflow/nmt2/nmt/data/hybrid2/'+prefix+'.'+str(vocab_size)+'.'+file,'w',encoding='utf-8') as fout:
                with open(in_path+prefix+'.'+file,'r',encoding='utf-8') as fin:
                    total_len = 0
                    for line in fin.readlines():
                        res = []
                        for token in line.strip('\n').split():
                            
                            if token not in vocab:
                                for char in token[:-1]:
                                    res.append(char)
                                    res.append('@@')
                                res.append(token[-1])
                                    
                            else:
                                res.append(token)
                        fout.write(' '.join(res)+'\n')
                        sen_len_counter[len(res)] += 1
                        total_len += 1
                    print(total_len)
            
            if prefix == 'traind':
                print('items_per_sentence: %s %s' % (file,vocab_size))
                sorted_length = sorted(sen_len_counter.items(), key = lambda x:x[0])
                total_counter = 0
                line_str = ''
                for lens,counter in sorted_length:
                    total_counter += counter
                    line_str += '%d %.2f ' % (lens, total_counter/total_len)
                print (line_str)

for file in ['message']:
    for vocab_size in [20000,40000,80000]:
        vocab = set()
        with open('/ldev/tensorflow/nmt2/nmt/data/hybrid2/vocab.'+str(vocab_size)+'.'+file,'r+',encoding='utf-8') as fin:
            for line in fin.readlines():
                vocab.add(line.strip())
        for prefix  in ['train','test','dev']:
            sen_len_counter = defaultdict(int)
            with open('/ldev/tensorflow/nmt2/nmt/data/hybrid2/'+prefix+'.'+str(vocab_size)+'.'+file,'w',encoding='utf-8') as fout:
                with open(in_path+prefix+'.'+file,'r',encoding='utf-8') as fin:
                    total_len = 0
                    for line in fin.readlines():
                        res = []
                        for token in line.strip('\n').split():
                                res.append(token)
                        fout.write(' '.join(res)+'\n')
                        sen_len_counter[len(res)] += 1
                        total_len += 1
                    print(total_len)
            
            if prefix == 'traind':
                print('items_per_sentence: %s %s' % (file,vocab_size))
                sorted_length = sorted(sen_len_counter.items(), key = lambda x:x[0])
                total_counter = 0
                line_str = ''
                for lens,counter in sorted_length:
                    total_counter += counter
                    line_str += '%d %.2f ' % (lens, total_counter/total_len)
                print (line_str)

3748600
20000
20000
3748600
20000
20000
3748600
20000
20000
3748600
20000
20000
3748600
20000
20000
3748600
20000
20000


# Segment

In [12]:
# Configs including a end
seg_len = 5
seg_end = '<#>'
seg_pad = '<_>'
seg_separator = '\t'
seg_inter_separator = ' '
vocab_nums = [40000]
from collections import defaultdict
# read the vocabs and add all characters in the vocab
def add_chars_to_vocab(file_path,vocab_path, word_num = -1):
    with open(file_path,'r+',encoding='utf-8') as fin:
        with open(vocab_path+'_tmp','w+',encoding='utf-8') as fout:
            lines = fin.readlines()
            print('original num: %d' % len(lines))
            vocab = defaultdict(int)
            vocab[seg_end] = 999
            vocab[seg_pad] = 999
            
            # first 3 lines are special tokens
            if word_num != -1:
                lines = lines[0:0+word_num]
            else:
                lines = lines[0:]
            for line in lines:
                line = line.strip('\n')
                line=line.replace(' ','')
                line=line.replace('\t','')
                #vocab.add(word)
                if len(line) > 1:
                    for char in line:
                        vocab[char] += 1
                
            for token in vocab.keys():
                if vocab[token] > 0:
                    fout.write(token+'\n')
                #print(token)
            print('new num: %d' % len(vocab))
            
    with open(vocab_path+'_tmp','r+',encoding='utf-8') as fin:
        with open(vocab_path+'_seg','w+',encoding='utf-8') as fout:
            lines = fin.readlines()
            print('original num: %d' % len(lines))
            vocab = set()
            vocab.add(seg_end)
            vocab.add(seg_pad)
            
            # first 3 lines are special tokens
            if word_num != -1:
                lines = lines[0:0+word_num]
            else:
                lines = lines[0:]
            for line in lines:
                line = line.strip('\n')
                #vocab.add(word)
                if len(line) > 0:
                    for char in line:
                        vocab.add(str(char))
            count = 0
            for token in vocab:
                if len(token) > 0:
                    fout.write(token+'\n')
                    count += 1
                #print(token)
            print('new num: %d' % count)



def convert_to_seg_file(file_path, seg_len,vocab):
    with open(file_path, 'r+', encoding='utf-8') as fin:
        nfile_path = file_path
        with open(nfile_path + '_seg', 'w+', encoding='utf-8') as fout,open(nfile_path + '_seg_len', 'w+', encoding='utf-8') as flout:
            lines = fin.readlines()
            for line in lines:
                items = line.strip('\n').split(' ')
                seg_items = []
                for item in items:
                    item = list(item)
                    item = item[0:seg_len - 1]
                    item.append(seg_end)
                    while len(item) != seg_len:
                        item.append(seg_pad)
                    seg_items.append(seg_inter_separator.join(item))
                flout.write(' '.join([str(min(seg_len,len(x)+1)) for x in items]) + '\n')
                fout.write(seg_separator.join(seg_items) + '\n')

for vocab in vocab_nums:
    # TODO 所有
    add_chars_to_vocab(r'/ldev/tensorflow/nmt2/nmt/data/hybrid2/train.%d.message' % (vocab), r'/ldev/tensorflow/nmt2/nmt/data/hybrid2/vocab.%d.message' % vocab)
    add_chars_to_vocab(r'/ldev/tensorflow/nmt2/nmt/data/hybrid2/train.%d.response'% (vocab), r'/ldev/tensorflow/nmt2/nmt/data/hybrid2/vocab.%d.response'% vocab)
    convert_to_seg_file(r'/ldev/tensorflow/nmt2/nmt/data/hybrid2/train.%d.message'% (vocab), seg_len,vocab)
    convert_to_seg_file(r'/ldev/tensorflow/nmt2/nmt/data/hybrid2/train.%d.response'% (vocab), seg_len,vocab)
    convert_to_seg_file(r'/ldev/tensorflow/nmt2/nmt/data/hybrid2/test.%d.message'% (vocab), seg_len,vocab)
    convert_to_seg_file(r'/ldev/tensorflow/nmt2/nmt/data/hybrid2/test.%d.response'% (vocab), seg_len,vocab)
    convert_to_seg_file(r'/ldev/tensorflow/nmt2/nmt/data/hybrid2/dev.%d.message'% (vocab), seg_len,vocab)
    convert_to_seg_file(r'/ldev/tensorflow/nmt2/nmt/data/hybrid2/dev.%d.response'% (vocab), seg_len,vocab)




original num: 3748600
new num: 6690
original num: 6690
new num: 6690
original num: 3748600
new num: 10552
original num: 10552
new num: 10552
